In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle


In [8]:
#Load the dataset
df = pd.read_csv('Churn_Modelling.csv')
df.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
## Delete the irrelevent columns
df = df.drop(['RowNumber','CustomerId','Surname'],axis=1)


In [10]:
## There are two categorical veriables
## For gender I'll use label encoding
label_encode_gender = LabelEncoder()
df['Gender'] = label_encode_gender.fit_transform(df['Gender'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [11]:
## For Geography I'll use onehotencoder
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder(sparse_output=False,handle_unknown='ignore')
geo_encoder = onehot_encoder_geo.fit_transform(df[['Geography']])
geo_encoder 
import numpy as np
cols = np.array(onehot_encoder_geo.get_feature_names_out(['Geography']))

In [13]:
geo_encoder_df = pd.DataFrame(geo_encoder,columns=cols)
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [14]:
## Combine all the columns in df
df = pd.concat([df.drop('Geography',axis=1),geo_encoder_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [17]:
### Now divide the dataset into dependent and independent features
X= df.drop('EstimatedSalary',axis=1)
y= df['EstimatedSalary']

##Split into train test split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

## Scaling the features

scaler = StandardScaler()
X_train =  scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [18]:
## Save the encoders
## Gender Encoder
with open('label_encoder_gender_reg.pkl', 'wb') as file:
    pickle.dump(label_encode_gender,file)

## Onehotencoder for geography

with open('onehot_encoder_geo_reg.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

## Save the scaler

with open('scaler_reg.pkl','wb') as file:
    pickle.dump(scaler,file)

In [19]:
## ANN with Regression 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime
 

In [20]:


# Define the model
model = Sequential()

# Specify the input layer using Input()
model.add(Input(shape=(X_train.shape[1],)))  # Replace 'input_size' with the actual number of features

# Add additional layers
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

# Output layer
model.add(Dense(1))  # For binary classification

In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
## Compile the model for forward propagation 

model.compile(optimizer='adam',loss="mean_absolute_error",metrics=['mae'])

In [24]:
## Setup the tensor board for logs
log_dir ="regressionlogs/fit"+datetime.datetime.now().strftime("%y%m%d-%H%M%S")
 ## setup callback to visualize all your logs
tensorflow_callback= TensorBoard(log_dir=log_dir,histogram_freq=1)

In [25]:
## setup for early stoping for epochs
early_stoping_callback= EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [26]:
### Trainig the model
history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),
    epochs=100,callbacks=[tensorflow_callback,early_stoping_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - loss: 100652.4062 - mae: 100652.4062 - val_loss: 98489.7266 - val_mae: 98489.7266
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 99138.1328 - mae: 99138.1328 - val_loss: 96810.3047 - val_mae: 96810.3047
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 97007.8750 - mae: 97007.8750 - val_loss: 92562.0000 - val_mae: 92562.0000
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 92606.5625 - mae: 92606.5625 - val_loss: 85577.5312 - val_mae: 85577.5312
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 85524.7734 - mae: 85524.7734 - val_loss: 76783.6016 - val_mae: 76783.6016
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 76786.5078 - mae: 76786.5078 - val_loss: 67833.2031 - val_mae: 67833.2031
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 66709.8125 - mae: 66709.8125 - val_loss: 60307.5430 - val_mae: 60307.5430
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/ste

In [27]:
%load_ext tensorboard

In [28]:
%tensorboard --logdir regressionlogs/fit241009-145413

In [31]:
## Evaluate the model
test_loss,test_mae = model.evaluate(X_test,y_test)
print(f'Test Mae:{test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 51057.5938 - mae: 51057.5938
Test Mae:50212.82421875


In [32]:
model.save('regression_model.h5')